In [1]:
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time
import random
import spacy
import torch.nn.init as init

torch.backends.cudnn.deterministic = True

### 基础设定(Hyper Parameters)

In [2]:

VOCABULARY_SIZE = 5000
LEARNING_RATE = 0.2
BATCH_SIZE = 100
NUM_EPOCHS = 15
MAX_LEN = 380

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.text
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.label.size(0)
            correct_pred += (predicted_labels == batch_data.label.long()).sum()
        return correct_pred.float()/num_examples * 100

### 加载数据

In [4]:
# TEXT = data.Field(tokenize = 'spacy')
TEXT = data.Field(sequential=True, tokenize = 'spacy',fix_length=MAX_LEN) # 为了 packed_padded_sequence
LABEL = data.LabelField(dtype = torch.long)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
# train_data, valid_data = train_data.split(random_state=random.seed(RANDOM_SEED),
#                                           split_ratio=0.8)
print(f'Num Train: {len(train_data)}')
# print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')


Num Train: 25000
Num Test: 25000


In [5]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 5002
Number of classes: 2


In [6]:
train_loader, test_loader = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size=BATCH_SIZE,
    device=DEVICE)

In [7]:
for batch in train_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

Text matrix size: torch.Size([380, 100])
Target vector size: torch.Size([100])

Test:
Text matrix size: torch.Size([380, 100])
Target vector size: torch.Size([100])


### 创建模型

In [8]:
'''
LSTM(
  (embedding): Embedding(5002, 50)
  (lstm): LSTM(50, 64, num_layers=2, dropout=0.2)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)
'''

from torch import nn
class LSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim):
        
        super(LSTM,self).__init__()
        self.embedding = nn.Embedding(len(TEXT.vocab), embedding_dim)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim, 
            num_layers=2,          
            batch_first=False,  #(squence length,batch_size)
            dropout=0.2
        )
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        
    def forward(self, text):
        # text shape (squence length, batch_size)
        # embedding shape (squence length, batch_size, embedding_dim)
        # output shape (squence length,batch_size, output_size)
        embedded = self.embedding(text)
        output, hidden = self.lstm(embedded)
#         output = self.relu(output)
        out = self.fc(output[-1, :, :])
        return out

In [9]:
def weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.constant_(m.bias, 0)
    # 也可以判断是否为conv2d，使用相应的初始化方式 
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

In [10]:
# INPUT_DIM = len(TEXT.vocab)
# INPUT_DIM = 1
EMBEDDING_DIM = 50
HIDDEN_DIM = 64
OUTPUT_DIM = 2

model = LSTM(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
model.apply(weight_init)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

### 训练模型

In [11]:
# 计算准确率
def binary_accuracy(preds, y):
    count = 0
    for index in range(len(y)):
        if torch.argmax(preds[index]) == y[index]:
            count = count + 1
#     rounded_preds = torch.round(preds)  # 把概率的结果 四舍五入
#     correct = (rounded_preds == y).float()  # True False -> 转为 1， 0
    acc = count / len(y)
    return acc

In [14]:
print(model)
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    optimizer.zero_grad()
    for batch_idx, batch_data in enumerate(train_loader):

        logits = model(batch_data.text)
        cost = loss_func(logits, batch_data.label)
        acc = binary_accuracy(logits, batch_data.label)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.4f}')
            prediction = torch.max(logits, 1)[1]
            pred_y = prediction.data.numpy()
            target_y = batch_data.label.data.numpy()
            accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
            print('accuracy: ',accuracy)


    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_loader, DEVICE):.2f}%')

LSTM(
  (embedding): Embedding(5002, 50)
  (lstm): LSTM(50, 64, num_layers=2, dropout=0.2)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)
Epoch: 001/015 | Batch 000/250 | Cost: 0.6884
accuracy:  0.55
Epoch: 001/015 | Batch 050/250 | Cost: 0.7133
accuracy:  0.47
Epoch: 001/015 | Batch 100/250 | Cost: 0.6912
accuracy:  0.52
Epoch: 001/015 | Batch 150/250 | Cost: 0.6959
accuracy:  0.52
Epoch: 001/015 | Batch 200/250 | Cost: 0.6932
accuracy:  0.49


ValueError: too many values to unpack (expected 2)

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
print('Probability positive:')
predict_sentiment(model, "I really love this movie. This movie is so great!")